Importing the Libraries

In [ ]:
# importing the necessary libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPool1D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import time
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

BERTweet Loader

In [ ]:
# Load BERTweet model and tokenizer
bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

# Function to tokenize and get embeddings for a list of sentences
# the sentence to be tokenized would be the input of the model
def get_bertweet_embeddings(sentences):
    input_ids = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)["input_ids"]
    with torch.no_grad():
        outputs = bertweet(input_ids)
    return outputs.last_hidden_state.mean(dim=1).numpy()  # Using mean pooling

Loading Dataset

In [3]:
# Load your dataset
# Give the path to your dataset here
df = pd.read_csv("/kaggle/input/test-set-adobe/behaviour_simulation_test_company.csv",encoding='latin-1')


In [7]:
# make a column called tweer which would contain the content to be tokenized
# here we're tokenizing the username
# we have used this code repeatedly to generate the embeddings by changing the path and the column name
df['tweet']=df['username']

In [8]:
# A code to remove invalid entries in the dataframe
# Commented out as we've already corrected the dataset


# df['likes'] = pd.to_numeric(df['likes'], errors='coerce')

# # Drop rows where 'likes' is NaN (non-numeric values)
# df = df.dropna(subset=['likes'])

# # Convert 'likes' column back to integers
# df['likes'] = df['likes'].astype(int)

In [10]:
# converting the tweet column into string type
df['tweet'] = df['tweet'].astype(str)

Tokenizing Content

In [ ]:
# Assuming you have already defined the get_bertweet_embeddings function

# Define the batch size
batch_size = 1000  # You can adjust this based on your available memory

# Process the test data in chunks
start = time.time()
print("train data embeddings started")
for start_idx in range(0, len(df), batch_size):
    end_idx = start_idx + batch_size
    print(start_idx)
    batch_sentences = df["tweet"].iloc[start_idx:end_idx].tolist()
    X_batch = get_bertweet_embeddings(batch_sentences)
    
    # Concatenate the batch embeddings to the array
    if start_idx == 0:
        X_test = X_batch
#         y_test_batch = df["likes"].iloc[start_idx:end_idx].values
    else:
        X_test = np.concatenate([X_test, X_batch])
#         y_test_batch = np.concatenate([y_test_batch, df["likes"].iloc[start_idx:end_idx].values])

# Save the test embeddings to a file
np.save("content-test-company-username.npy", X_test)

print(f"Time required = {time.time() - start}")
print("test data embeddings ended")